In [ ]:
import os
from tqdm import tqdm
import pandas as pd


In [ ]:
from dotenv import load_dotenv
import sys
sys.path.append("../scripts")
from bookFetch  import pdfParser
load_dotenv("../var.env")
parse = pdfParser()

In [ ]:
from concurrent.futures import ThreadPoolExecutor
pdf_folder = "../pdfs"
filepaths = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder) if f.endswith(".pdf")]

all_entries = []


with ThreadPoolExecutor(max_workers=4) as executor:
    for result in tqdm(executor.map(parse.parse_pdf, filepaths), total=len(filepaths)):
            if result:
                all_entries.extend(result)

In [ ]:
books_data = pd.DataFrame([e for e in all_entries if e is not None])
books_data.to_csv("../data/raw/nyt_bestsellers_data.csv")

In [18]:
books_data = pd.read_csv("../data/raw/nyt_bestsellers_data.csv", low_memory=False)
unique_books = books_data[["title", "author"]].drop_duplicates().reset_index()

In [7]:
all_entries = list(unique_books.to_dict(orient="records"))

In [ ]:
def get_season(date):
    month = date.month
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    elif month in [9, 10, 11]:
        return "Autumn"

In [ ]:
books_data.dropna(how="all", inplace=True)

In [ ]:
books_data["date"] = pd.to_datetime(books_data["date"])

In [ ]:
books_data["season"] = books_data["date"].apply(get_season)

In [ ]:
books_data.to_csv("../data/raw/nyt_bestsellers.csv")

In [9]:
api_call_count = 0

In [13]:
from more_itertools import chunked
if os.path.exists("../data/raw/batch_index.txt"):
    with open("../data/raw/batch_index.txt", "r") as f:
        start_batch = int(f.read())
else:
    start_batch = 0    
batches = list(chunked(all_entries, 1000))
for i, batch in enumerate(batches[start_batch:], start=start_batch):
    result = parse.book_detail_generator(batch)
    if result == "Stop":
        with open("../data/raw/batch_index.txt", "w") as f:
            f.write(str(i))
        break

  0%|          | 0/646 [00:00<?, ?it/s]


In [20]:
unique_books.shape

(8646, 3)

In [19]:
unique_books[unique_books["title"] == "Fair Game"]

,index,title,author
2164,12584,Fair Game,Valerie Plame Wilson
3811,19628,Fair Game,Patricia Briggs
